In [1]:
!pip install transformers datasets

In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [3]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.0 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset
import numpy as np

In [5]:
# The Recognizing Textual Entailment (RTE) datasets come from a series of annual
# textual entailment challenges. We combine the data from RTE1 (Dagan et al.,
# 2006), RTE2 (Bar Haim et al., 2006), RTE3 (Giampiccolo et al., 2007), and RTE5
# (Bentivogli et al., 2009).4 Examples are constructed based on news and
# Wikipedia text. We convert all datasets to a two-class split, where for
# three-class datasets we collapse neutral and contradiction into not
# entailment, for consistency.
raw_datasets = load_dataset("glue", "rte")

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/584k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/69.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/621k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2490 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/277 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [6]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 2490
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 277
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3000
    })
})

In [7]:
raw_datasets['train'].features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['entailment', 'not_entailment'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [8]:
raw_datasets['train']['sentence1'][:10]

['No Weapons of Mass Destruction Found in Iraq Yet.',
 'A place of sorrow, after Pope John Paul II died, became a place of celebration, as Roman Catholic faithful gathered in downtown Chicago to mark the installation of new Pope Benedict XVI.',
 'Herceptin was already approved to treat the sickest breast cancer patients, and the company said, Monday, it will discuss with federal regulators the possibility of prescribing the drug for more breast cancer patients.',
 'Judie Vivian, chief executive at ProMedica, a medical service company that helps sustain the 2-year-old Vietnam Heart Institute in Ho Chi Minh City (formerly Saigon), said that so far about 1,500 children have received treatment.',
 "A man is due in court later charged with the murder 26 years ago of a teenager whose case was the first to be featured on BBC One's Crimewatch. Colette Aram, 16, was walking to her boyfriend's house in Keyworth, Nottinghamshire, on 30 October 1983 when she disappeared. Her body was later found i

In [9]:
raw_datasets['train']['sentence2'][:10]

['Weapons of Mass Destruction Found in Iraq.',
 'Pope Benedict XVI is the new leader of the Roman Catholic Church.',
 'Herceptin can be used to treat breast cancer.',
 'The previous name of Ho Chi Minh City was Saigon.',
 'Paul Stewart Hutchinson is accused of having stabbed a girl.',
 'Bakri was briefly detained, but was released.',
 'Three quarters of U.S. illegal immigrants have children.',
 'Aristide had Prime Minister Robert Malval  murdered in Port-au-Prince.',
 'Dick Cheney was the Vice President of Bush.',
 "Microsoft's Hotmail has raised its storage capacity to 250MB."]

In [10]:
checkpoint = 'distilbert-base-cased'

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
  Trainer, TrainingArguments

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
tokenizer(
    raw_datasets['train']['sentence1'][0],
    raw_datasets['train']['sentence2'][0]
)

{'input_ids': [101, 1302, 20263, 1104, 8718, 14177, 17993, 17107, 1107, 5008, 6355, 119, 102, 20263, 1104, 8718, 14177, 17993, 17107, 1107, 5008, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
result = _

In [15]:
# distilbert doesn't use token_type_ids
result.keys()

dict_keys(['input_ids', 'attention_mask'])

In [16]:
tokenizer.decode(result['input_ids'])

'[CLS] No Weapons of Mass Destruction Found in Iraq Yet. [SEP] Weapons of Mass Destruction Found in Iraq. [SEP]'

In [17]:
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
training_args = TrainingArguments(
  output_dir='training_dir',
  eval_strategy='epoch',
  save_strategy='epoch',
  num_train_epochs=5,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=64,
  logging_steps=150, # otherwise, 'no log' will appear under training loss
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [19]:
from evaluate import load

In [20]:
metric = load("glue", "rte")

In [21]:
metric.compute(predictions=[1, 0, 1], references=[1, 0, 0])

{'accuracy': 0.6666666666666666}

In [22]:
from sklearn.metrics import f1_score

In [23]:
def compute_metrics(logits_and_labels):
  logits, labels = logits_and_labels
  predictions = np.argmax(logits, axis=-1)
  acc = np.mean(predictions == labels)
  f1 = f1_score(labels, predictions)
  return {'accuracy': acc, 'f1': f1}

In [24]:
def tokenize_fn(batch):
  return tokenizer(batch['sentence1'], batch['sentence2'], truncation=True)

In [25]:
tokenized_datasets = raw_datasets.map(tokenize_fn, batched=True)

Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

Map:   0%|          | 0/277 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [26]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [27]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.696100,0.691549,0.527076,0.000000
2,0.658500,0.684881,0.584838,0.590747
3,0.444400,0.814758,0.555957,0.568421
4,0.222200,1.287232,0.599278,0.577947
5,0.110700,1.603193,0.624549,0.573770


TrainOutput(global_step=780, training_loss=0.41216052296834116, metrics={'train_runtime': 124.569, 'train_samples_per_second': 99.945, 'train_steps_per_second': 6.262, 'total_flos': 542121276647352.0, 'train_loss': 0.41216052296834116, 'epoch': 5.0})

In [28]:
trainer.save_model('my_saved_model')

In [29]:
from transformers import pipeline

p = pipeline('text-classification', model='my_saved_model', device=0)

In [30]:
p({'text': 'I went to the store', 'text_pair': 'I am a bird'})

{'label': 'LABEL_1', 'score': 0.7851224541664124}